In [1]:
from secret_key import secret_key_1
import os
from langchain_openai import OpenAI

In [2]:
os.environ['OPENAI_API_KEY'] = secret_key_1

In [3]:
llm = OpenAI(temperature=0.6)


In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

ipl_category = PromptTemplate(
    input_variables = ['category'],
    template = "Give me a sports trivia about Indian Premier League {category} with options. I'll give you an answer for the same."
)

category_chain = LLMChain(llm=llm, prompt = ipl_category, output_key = "Question")

ans_prompt = PromptTemplate(
    input_variables = ['option_selected'],
    template = "Is {option_selected} is the right answer?"
)

ans_chain = LLMChain(llm=llm, prompt = ans_prompt, output_key = "User_Answer")

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


ipl_category = PromptTemplate(
    input_variables = ['category'],
    template = "Give me a sports trivia about Indian Premier League {category} with options. I'll give you an answer for the same."
)

category_chain = LLMChain(llm=llm, prompt = ipl_category, output_key = "category_selected")

ans_prompt = PromptTemplate(
    input_variables = ['category_selected'],
    template = "Is {category_selected} is the right answer?"
)

ans_chain = LLMChain(llm=llm, prompt = ans_prompt, output_key = "user_answer")

prompt_chain = SequentialChain(chains = [category_chain, ans_chain],
                              input_variables = ['category'],
                              output_variables = ['category_selected', "user_answer"]
)

prompt_chain.invoke({'category':'Bowler'})

{'category': 'Bowler',
 'category_selected': '\n\nQuestion: Who is the only bowler in Indian Premier League history to take a hat-trick in the first over of a match?\n\nA) Lasith Malinga\nB) Amit Mishra\nC) Pravin Tambe\nD) Andrew Tye\n\nAnswer: C) Pravin Tambe',
 'user_answer': ''}

In [100]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm = llm,
    memory = memory
)

category_input = "Bowler"
team_input = "Royal Challengers Banglore"

que_prompt = PromptTemplate(
    input_variables = ['category_input', 'team_input'],
    template = "Generate one sports quiz related to Indian Premier League related to a {category_input} from {team_input} team and provide 4 options of possible answer. Only give question and 4 option as the output, don't give any other thing in the output. Don't give the answer, I'll give the answer"
)
que_chain = LLMChain(llm=llm, prompt=que_prompt, output_key='question')

que_prompt.format(category_input=category_input,team_input=team_input)



"Generate one sports quiz related to Indian Premier League related to a Bowler from Royal Challengers Banglore team and provide 4 options of possible answer. Only give question and 4 option as the output, don't give any other thing in the output. Don't give the answer, I'll give the answer"

In [101]:
convo.run(que_prompt.format(category_input=category_input,team_input=team_input))

' Sure, here is your sports quiz question: Which bowler from the Royal Challengers Bangalore team has the most wickets in the Indian Premier League? A) Yuzvendra Chahal B) Umesh Yadav C) Mohammed Siraj D) Navdeep Saini'

In [96]:
convo.run('Yuzvendra Chahal')


' That is correct! Yuzvendra Chahal holds the record for the most wickets in a single IPL season for Royal Challengers Bangalore with 21 wickets in the 2019 season. Well done!'

In [98]:
convo.memory.clear()

In [99]:
convo.memory.buffer

''

In [102]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=2)

convo = ConversationChain(
    llm = llm,
    memory = memory
)

category_input = "Bowler"
team_input = "Royal Challengers Banglore"

que_prompt = PromptTemplate(
    input_variables = ['category_input', 'team_input'],
    template = "Generate one sports quiz related to Indian Premier League related to a {category_input} from {team_input} team. Only give question, don't give any other thing in the output. Don't give the answer, I'll give the answer"
)
que_chain = LLMChain(llm=llm, prompt=que_prompt, output_key='question')

que_prompt.format(category_input=category_input,team_input=team_input)



"Generate one sports quiz related to Indian Premier League related to a Bowler from Royal Challengers Banglore team. Only give question, don't give any other thing in the output. Don't give the answer, I'll give the answer"

In [103]:
convo.run(que_prompt.format(category_input=category_input,team_input=team_input))

' Sure, here is a question for you: Which bowler from the Royal Challengers Bangalore team has the most wickets in the Indian Premier League?'

In [104]:
convo.run('Provide option for the above questions')

' The options are: A) Yuzvendra Chahal, B) Umesh Yadav, C) Mohammed Siraj, D) Navdeep Saini.'

In [105]:
convo.run('Umesh Yadav')

' That is incorrect. The correct answer is Yuzvendra Chahal, who has taken 121 wickets in the IPL for the Royal Challengers Bangalore team. Would you like another question?'

In [107]:
theme = """
    I want you to help me making a quiz game. following in this conversation, I will provide you a category and a team,
    I want you to generate a quiz for that category and team of Indian Premier League. Here, for the first response,
    I only want you to give question or quiz, not any option or hint untill asked for. 

    Then in the next prompt, I will ask for 4 options for the same. Here I only want you to return
    options in following manner:
    A)
    B)
    C)
    D)

    And for the last prompt I will try to answer the quiz then you can verify it.

"""
convo.run(theme)

' Sure, I would be happy to help you create a quiz game for the Indian Premier League. Please provide me with a category and a team to generate a quiz for.'

In [108]:
convo.run('Bowler, Chennai Super Kings')

' Here is your quiz question: Who is the leading wicket-taker for the Chennai Super Kings team in the Indian Premier League?'

In [109]:
convo.run("provide opitons")

' A) Dwayne Bravo\nB) Ravindra Jadeja\nC) Imran Tahir\nD) Harbhajan Singh'

In [110]:
convo.run("answer is A")

' Sorry, that is incorrect. The correct answer is Dwayne Bravo with 147 wickets in 120 matches.'

In [116]:
import regex as re

t = """
Here is a question for the category "Team" and team "Delhi Capitals":

Which player has taken the most catches for Delhi Capitals in the Indian Premier League?

Remember, don't worry about the options yet, I will provide those in the next prompt. Just focus on the question for now.
"""

pattern = r':(.*?\?)'

# Extract data using regex
matches = re.findall(pattern, t, re.DOTALL)

if matches:
    for match in matches:
        print("Extracted data:", match.strip())


Extracted data: Which player has taken the most catches for Delhi Capitals in the Indian Premier League?
